In [8]:
import numpy as np
from scipy.optimize import linprog

In [9]:
class DantzigWolfe:
    """
    Implementa a Decomposição de Dantzig-Wolfe para problemas de otimização linear
    com estrutura de blocos angular.

    Esta versão foi refatorada para maior clareza e manutenibilidade.
    """

    def __init__(self, master_b, subproblems, tol=1e-7, max_iter=100):
        """
        Inicializa o solver Dantzig-Wolfe.

        Args:
            master_b (list or np.array): O vetor de recursos (lado direito) das restrições
                                        de acoplamento do problema mestre (vetor b).
            subproblems (list): Uma lista de dicionários, onde cada dicionário define um subproblema.
            tol (float): Tolerância para a verificação de otimalidade (custo reduzido >= -tol).
            max_iter (int): Número máximo de iterações do algoritmo.
        """
        self.master_b = np.array(master_b)
        self.subproblems = subproblems
        self.num_master_constraints = len(master_b)
        self.tol = tol
        self.max_iter = max_iter
        self.big_m = 1e6

        self.proposals = []
        self.rmp_columns = []
        self.rmp_costs = []
        
        print("Dantzig-Wolfe inicializado.")
        print(f"Número de restrições mestre: {self.num_master_constraints}")
        print(f"Número de subproblemas: {len(subproblems)}")

    def _initialize_with_artificials(self):
        """
        Inicializa o Problema Mestre Restrito com variáveis artificiais para
        garantir uma base factível inicial.
        """
        print("Inicializando o Problema Mestre com variáveis artificiais...")
        identity_basis = np.identity(self.num_master_constraints + 1)
        
        for i in range(self.num_master_constraints + 1):
            proposal = {
                'type': 'artificial',
                'cost': self.big_m,
                'column': identity_basis[:, i]
            }
            self._add_new_proposal(proposal)

    def _solve_master_problem(self):
        """
        Resolve o Problema Mestre Restrito (PMR) atual.

        Retorna:
            scipy.optimize.OptimizeResult: O objeto de resultado do linprog.
        """
        c_rmp = np.array(self.rmp_costs)
        A_eq_rmp = np.column_stack(self.rmp_columns)
        b_eq_rmp = np.append(self.master_b, 1)

        rmp_result = linprog(c=c_rmp, A_eq=A_eq_rmp, b_eq=b_eq_rmp, method='highs')
        
        return rmp_result

    def _solve_pricing_subproblems(self, duals):
        """
        Resolve os subproblemas de pricing usando os preços duais do mestre.

        Args:
            duals (np.array): O vetor de variáveis duais do problema mestre.

        Returns:
            tuple: Uma tupla contendo (custo_reduzido_minimo, melhor_proposta).
                   Retorna (float('inf'), None) se nenhum subproblema for resolvido.
        """
        lambda_dual = duals[:-1]
        pi_dual = duals[-1]
        
        min_reduced_cost = float('inf')
        best_new_proposal = None
        
        for j, sub in enumerate(self.subproblems):
            c_sub_pricing = sub['c'] - sub['A_master'].T @ lambda_dual
            
            sub_result = linprog(c=c_sub_pricing, 
                                 A_ub=sub.get('A_sub'), 
                                 b_ub=sub.get('b_sub'),
                                 bounds=sub.get('bounds'),
                                 method='highs')

            if not sub_result.success:
                print(f"Aviso: Subproblema {j} não foi resolvido com sucesso.")
                continue
            
            reduced_cost = sub_result.fun - pi_dual

            if reduced_cost < min_reduced_cost:
                min_reduced_cost = reduced_cost
                best_new_proposal = {
                    'type': 'real',
                    'x_k': sub_result.x,
                    'subproblem_index': j,
                    'cost': sub['c'] @ sub_result.x,
                    'column': np.append(sub['A_master'] @ sub_result.x, 1)
                }
                
        return min_reduced_cost, best_new_proposal

    def _add_new_proposal(self, proposal):
        """
        Adiciona uma nova proposta (coluna) ao Problema Mestre Restrito.

        Args:
            proposal (dict): Dicionário contendo as informações da nova proposta.
        """
        self.proposals.append(proposal)
        self.rmp_costs.append(proposal['cost'])
        self.rmp_columns.append(proposal['column'])
        if proposal['type'] == 'real':
             print(f"Adicionando nova proposta do subproblema {proposal['subproblem_index']}.")

    def solve(self):
        """
        Executa o algoritmo de Decomposição de Dantzig-Wolfe.
        Este método agora orquestra as chamadas para os métodos auxiliares.
        """
        self._initialize_with_artificials()

        for i in range(self.max_iter):
            print(f"\n--- Iteração {i+1} ---")

            # 1. Resolver o Problema Mestre
            rmp_result = self._solve_master_problem()
            if not rmp_result.success:
                print("ERRO: O Problema Mestre Restrito não pôde ser resolvido.")
                return None, None

            master_obj_val = rmp_result.fun
            duals = rmp_result.eqlin.marginals
            alphas = rmp_result.x
            
            print(f"Valor da Solução Mestre Atual: {master_obj_val:.6f}")
            print(f"Preços Duais (λ): {duals[:-1]}")
            print(f"Preço Dual (π): {duals[-1]:.4f}")
            
            # 2. Resolver os Subproblemas de Pricing
            min_reduced_cost, best_new_proposal = self._solve_pricing_subproblems(duals)
            print(f"Custo Reduzido Mínimo encontrado: {min_reduced_cost:.6f}")

            # 3. Verificação de Otimalidade
            if min_reduced_cost >= -self.tol:
                print("\nCondição de otimalidade atingida. Solução final encontrada.")
                final_solution = self._reconstruct_solution(alphas)
                final_value = master_obj_val
                return final_solution, final_value
            
            # 4. Adicionar Nova Coluna
            self._add_new_proposal(best_new_proposal)

        print("Número máximo de iterações atingido.")
        final_solution = self._reconstruct_solution(alphas)
        final_value = master_obj_val
        return final_solution, final_value

    def _reconstruct_solution(self, alphas):
        """
        Reconstrói a solução final para as variáveis originais x.
        """
        num_vars_per_sub = [len(sub['c']) for sub in self.subproblems]
        solutions_by_sub = [np.zeros(n) for n in num_vars_per_sub]

        print("\nReconstruindo a solução final...")
        print("Pesos (α) da solução final:")
        for i, alpha in enumerate(alphas):
            if alpha > self.tol:
                proposal = self.proposals[i]
                print(f"  - Proposta {i} (do subproblema {proposal.get('subproblem_index', 'Artificial')}): α = {alpha:.4f}")
                if proposal['type'] == 'real':
                    sub_idx = proposal['subproblem_index']
                    solutions_by_sub[sub_idx] += alpha * proposal['x_k']

        return np.concatenate(solutions_by_sub)

In [10]:
# Vetor b da restrição de acoplamento
master_b = [7]

# Lista de subproblemas (neste caso, apenas um)
subproblems_list = [
    {
        'c': np.array([-2, -3]),
        'A_master': np.array([[1, 2]]),
        'A_sub': np.array([[1, 0], [0, 1]]),
        'b_sub': np.array([2, 3]),
        'bounds': [(0, None), (0, None)] # x1 >= 0, x2 >= 0
    }
]

# --- Criando e resolvendo o problema ---
dw_solver = DantzigWolfe(master_b=master_b, subproblems=subproblems_list)
final_solution, final_value = dw_solver.solve()

# --- Exibindo os resultados ---
if final_solution is not None:
    print("\n\n================= RESULTADO FINAL =================")
    print(f"Valor Ótimo da Função Objetivo: {final_value:.6f}")
    print("Valores das Variáveis de Decisão:")
    print(f"  - Solução x: {final_solution}")
    print("===================================================")

Dantzig-Wolfe inicializado.
Número de restrições mestre: 1
Número de subproblemas: 1
Inicializando o Problema Mestre com variáveis artificiais...

--- Iteração 1 ---
Valor da Solução Mestre Atual: 8000000.000000
Preços Duais (λ): [1000000.]
Preço Dual (π): 1000000.0000
Custo Reduzido Mínimo encontrado: -9000013.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 2 ---
Valor da Solução Mestre Atual: 124988.625000
Preços Duais (λ): [-125001.625]
Preço Dual (π): 1000000.0000
Custo Reduzido Mínimo encontrado: -1000000.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 3 ---
Valor da Solução Mestre Atual: -11.375000
Preços Duais (λ): [-1.625]
Preço Dual (π): -0.0000
Custo Reduzido Mínimo encontrado: -0.750000
Adicionando nova proposta do subproblema 0.

--- Iteração 4 ---
Valor da Solução Mestre Atual: -11.500000
Preços Duais (λ): [-1.5]
Preço Dual (π): -1.0000
Custo Reduzido Mínimo encontrado: 0.000000

Condição de otimalidade atingida. Solução final encontrada.


In [11]:
# Vetor b das restrições de acoplamento
master_b = [6, 4]

# Lista de subproblemas
subproblems_list = [
    # Subproblema 1 (x1, x2)
    {
        'c': np.array([-2, -1]),
        'A_master': np.array([[1, 1], [0, 1]]),
        'A_sub': np.array([[1, 1], [0, 1]]),
        'b_sub': np.array([6, 2]),
        'bounds': [(0, None), (0, None)]
    },
    # Subproblema 2 (x3, x4)
    {
        'c': np.array([-3, -1]),
        'A_master': np.array([[1, 1], [2, 1]]),
        'A_sub': np.array([[-1, 1], [1, 1]]),
        'b_sub': np.array([3, 5]),
        'bounds': [(0, None), (0, None)]
    }
]

# --- Criando e resolvendo o problema ---
dw_solver = DantzigWolfe(master_b=master_b, subproblems=subproblems_list)
final_solution, final_value = dw_solver.solve()

# --- Exibindo os resultados ---
if final_solution is not None:
    print("\n\n================= RESULTADO FINAL =================")
    print(f"Valor Ótimo da Função Objetivo: {final_value:.6f}")
    print("Valores das Variáveis de Decisão:")
    num_vars_x1 = len(subproblems_list[0]['c'])
    solution_x1 = final_solution[:num_vars_x1]
    solution_x2 = final_solution[num_vars_x1:]
    print(f"  - Solução Bloco 1 (x1, x2): {solution_x1}")
    print(f"  - Solução Bloco 2 (x3, x4): {solution_x2}")
    print("===================================================")

Dantzig-Wolfe inicializado.
Número de restrições mestre: 2
Número de subproblemas: 2
Inicializando o Problema Mestre com variáveis artificiais...

--- Iteração 1 ---
Valor da Solução Mestre Atual: 11000000.000000
Preços Duais (λ): [1000000. 1000000.]
Preço Dual (π): 1000000.0000
Custo Reduzido Mínimo encontrado: -16000015.000000
Adicionando nova proposta do subproblema 1.

--- Iteração 2 ---
Valor da Solução Mestre Atual: 4599994.000000
Preços Duais (λ): [1000000.  -600001.5]
Preço Dual (π): 1000000.0000
Custo Reduzido Mínimo encontrado: -7000012.000000
Adicionando nova proposta do subproblema 0.

--- Iteração 3 ---
Valor da Solução Mestre Atual: 399986.800000
Preços Duais (λ): [1000000.    99999.7]
Preço Dual (π): -6000012.0000
Custo Reduzido Mínimo encontrado: -199997.400000
Adicionando nova proposta do subproblema 0.

--- Iteração 4 ---
Valor da Solução Mestre Atual: 249988.750000
Preços Duais (λ): [1000000.     124999.375]
Preço Dual (π): -6250008.7500
Custo Reduzido Mínimo encontr